In [3]:
import pandas as pd

df = pd.read_csv(r"C:\Users\misha\OneDrive\Desktop\DMV_CSV\sales_data.csv")

print(df.head())
    

       Date         Product  Units_Sold  Revenue
0  1/1/2023  Chocolate Cake          71      710
1  1/2/2023    Vanilla Cake          56      560
2  1/3/2023     Carrot Cake          47      470
3  1/4/2023      Red Velvet          68      680
4  1/5/2023         Brownie          79      790


In [4]:
import pandas as pd
# Load the dataset
df = pd.read_csv(r"C:\Users\misha\OneDrive\Desktop\DMV_CSV\sales_data.csv")

#Load the dataset into a DataFrame
sales_df = pd.DataFrame(df)



In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px



# Initialize the Dash app
app = dash.Dash(__name__)

# Define product options for dropdown menu
product_options = [{'label': product, 'value': product} for product in sales_df['Product'].unique()]

# Adding color mapping for products
color_map = {
    'Chocolate Cake': 'brown',
    'Vanilla Cake': '#B8860B',
    'Carrot Cake': 'orange',
    'Red Velvet': 'red',
    'Brownie': 'black'
}

# Define layout
app.layout = html.Div(children=[
    html.H1("Sales Dashboard", style={'color': 'white'}),
    
    # Dropdown menu to select a product
    dcc.Dropdown(
        id='product-dropdown',
        options=product_options,
        value=product_options[0]['value'],  # default value
        placeholder="Select a product"
    ),
    
    # Line chart to visualize the trend of units sold over time for the selected product
    dcc.Graph(id='line-chart'),
    
    # Bar chart to visualize total revenue generated from sales for each product
    dcc.Graph(id='bar-chart'),
    
    # Summary statistics for the 'Revenue' column
    html.Div(id='summary-stats')
])

# Callback to update line chart and bar chart based on selected product
@app.callback(
    [Output('line-chart', 'figure'),
     Output('bar-chart', 'figure')],
    [Input('product-dropdown', 'value')]
)
def update_charts(selected_product):
    # Filter dataframe for the selected product
    filtered_df = sales_df[sales_df['Product'] == selected_product]
    
    # Line chart for trend of units sold over time
    line_chart_fig = px.line(filtered_df, x='Date', y='Units_Sold', title=f'Units Sold Over Time for {selected_product}')
    line_chart_fig.update_traces(line_color=color_map[selected_product])
    
    # Bar chart for total revenue generated from sales for each product
    bar_chart_fig = px.bar(sales_df, x='Product', y='Revenue', color='Product', color_discrete_map=color_map, title='Total Revenue by Product')
    
    # To display the graphs
    return line_chart_fig, bar_chart_fig

# Callback to display summary statistics
@app.callback(
    Output('summary-stats', 'children'),
    [Input('product-dropdown', 'value')]
)
def update_summary_stats(selected_product):
    # Filter dataframe for the selected product
    filtered_df = sales_df[sales_df['Product'] == selected_product]
    
    # Calculate summary statistics
    mean_revenue = filtered_df['Revenue'].mean().round(2)
    median_revenue = filtered_df['Revenue'].median()
    min_revenue = filtered_df['Revenue'].min()
    max_revenue = filtered_df['Revenue'].max()
    
    # Display summary statistics
    return html.Div([
        html.H3('Summary Statistics for Revenue', style={'color': 'white'}),
        html.Table([
            html.Tr([html.Td('Mean:', style={'color': 'white'}), html.Td(mean_revenue, style={'color': 'white'})]),
            html.Tr([html.Td('Median:', style={'color': 'white'}), html.Td(median_revenue, style={'color': 'white'})]),
            html.Tr([html.Td('Min:', style={'color': 'white'}), html.Td(min_revenue, style={'color': 'white'})]),
            html.Tr([html.Td('Max:', style={'color': 'white'}), html.Td(max_revenue, style={'color': 'white'})])
        ])
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


: 